For each Wasm binary we have a three (non-simetric) space where the dimensions are: the mutator, the variants that the mutator generetes and the iteration of the mutation. Notice that the third dimension (the iteration of the mutator) also generates another space for the new mutation. This space can be represented as the following tree. 

The root of the tree is an arbitrary binary, and two levels below the binary, we reprenset the applicable mutations and the variants.


```

                binary
           /       |         \
dim0 ->   M1       M2         M3
         /  \    / |  \    /  |   \
dim1 -> v1  v2  v1 v2 v3  v1 ...  vn
         |
dim2 -> M1 ...
```

This tree offers some interesting properties.
- If the second or third level below the binary are empty, then the binary cannot be mutated
- If the order of the mutators are always the same, we can map an interesting variant with the applied mutators from the leaf to the root. 
- Theorethically speaking, the tree is always infinite, the reason is that our mutators can always generate bogus information.

Now, the question is how we generate the space. As we mentioned, some mutators generate infinite variants. Therefore, is impossible to generate the subtree for them. On the other hand, the tree has and infinite height, i.e. we can mutate the root binary as many times we needed (going down and down in the tree).

To solve the first issue we need to sample the mutators that generate an infinite number of variants.
For the second issue, we implement the construction of the tree for each variant up to two levels lazily. 



In the following list we enumerate the mutators, if they affect the execution (in terms of performance only, notice we only apply semantic equivalent transformations), if they increase/reduce the size of the variants (hint on the complexity of the variant),  a formula to calculate the number of variants they generate and how we can sample them.

| Mutator | Description | Affect execution | Affect size |  Number of variants |  Sample method |
|--       | --          |------     |----                 | ------         | ----        |
| PeepholeMutator| Creates an egraph based in rewriting rules for peephole transformations. In practice, this mutator has a fixed limit, usually on the height of the generated subtree.  | X | X | **oo** | Let **H** the maximum height, **M** the maximum number of trees that we can generate from the egraph (our **oo**) and **T** all the places valid for the generation of an egraph. The number of variants $$ |V| \le \Sigma_{1}^{T}\Sigma_{1}^{H}| egraph(H).trees. | $$ **This is extremely expensive $$ O(I\times M\times H) $$**  |
| CodemotionMutator| Unroll a **loop** or invert an **if** contruction. | | | \|loops\| + \|ifs\| | \|loops\| + \|ifs\||
| AddTypeMutator| Add a (1) random type to the type definition of the binary. | | | 1 (add a new type) | 1 |
| AddFunctionMutator| Add a new random function to the function and code sections. | | X | 1 | 1 |
| RemoveItemMutator(Item::Function)| Removes a random function (to be semantically equivalent, the function needs not to be exported or used |  | X | $$ \le |functions|$$ | $$ \le |functions|$$ |
| RemoveItemMutator(Item::Global)| Removes a global |  | X | $$ \le |globals| $$ | $$ \le |globals| $$ |
| RemoveItemMutator(Item::Memory)| Removes a memory |  | X | $$ \le |memories| $$ | $$ \le |memories| $$ |
| RemoveItemMutator(Item::Table)| Removes a table | | X | $$ \le |tables| $$ | $$ \le |tables| $$ |
| RemoveItemMutator(Item::Type)| Removes a type | | X | $$ \le |types| $$ | $$ \le |types| $$ |
| RemoveSection::Custom| Removes a custom section | | X | $$ \le |customs| $$ | $$ \le |customs| $$ |
| CustomSectionMutator| Changes the data or the name of the custom section | | X | $$ \le 2|customs| $$ | $$ \le 2|customs| $$ |
| RemoveSection::Empty| Removes an empty section | | X | $$ \le 2|empty| $$ | $$ \le 2|empty| $$ |


We built a tool able to generate the two dimensions of the mutation space for a given binary.

Given a binary, this tools provides the following info:
    
```json

"id": "59955b4c538d0b45961e19b70c4ab7a101968561a353186045bde628c0d72dbb.wasm",
    ...
    "mutations": [
      {
        "class_name": "PeepholeMutator::new(10)",
        ...
        "map": [
            ... Where to apply it
        ],
      },
      {
        "class_name": "CodemotionMutator",
        ...
        "map": [
            ... Where to apply it
        ],
      },
      {
        "class_name": "AddTypeMutator { max_params: 20, max_results: 20 }",
        ...
        "map": [
            ... Where to apply it
        ],
      },
      {
        "class_name": "AddFunctionMutator",
        ...
        "map": [
            ... Where to apply it
        ],
      },
      {
        "class_name": "RemoveItemMutator(Item::Function)",
        ...
        "map": [
            ... Where to apply it
        ],
      },
      {
        "class_name": "RemoveItemMutator(Item::Global)",
        ...
        "map": [
            ... Where to apply it
        ],
      },
      {
        "class_name": "RemoveItemMutator(Item::Memory)",
        ...
        "map": [
            ... Where to apply it
        ],
      },
      {
        "class_name": "RemoveItemMutator(Item::Table)",
        ...
        "map": [
            ... Where to apply it
        ],
      },
      {
        "class_name": "RemoveItemMutator(Item::Type)",
        ...
        "map": [
            ... Where to apply it
        ],
      }

```

Given this info and the previous presented table, we can answer for example, how many variants this binary will generate with one iteration of the mutators. This can be calculated using the *sample method* of the table. In practice we would only need count the size of the `map` property in the previous json.

In [ ]:
! RUST_LOG=analyzer,wasm_mutate=debug  ../analyzer/target/debug/analyzer  admin admin  extract -r -d 5 --input  test.wasm > meta.json

[2022-08-01T09:19:22Z DEBUG analyzer] Reseting 
[2022-08-01T09:19:22Z DEBUG analyzer] Extracting...
[2022-08-01T09:19:22Z DEBUG analyzer::subcommands::extract] Final files count 1
[2022-08-01T09:19:22Z DEBUG analyzer::subcommands::extract] 
    Extracting test.wasm 
[2022-08-01T09:19:22Z DEBUG wasm_mutate::mutators::peephole] Getting info for 70 functions
[2022-08-01T09:19:53Z WARN  wasm_mutate::mutators::peephole::dfg] Wasm operator not implemented: CallIndirect { index: 0, table_index: 0, table_byte: 0 }
[2022-08-01T09:19:53Z WARN  wasm_mutate::mutators::peephole::dfg] Wasm operator not implemented: CallIndirect { index: 0, table_index: 0, table_byte: 0 }
[2022-08-01T09:19:54Z WARN  wasm_mutate::mutators::peephole::dfg] Wasm operator not implemented: CallIndirect { index: 1, table_index: 0, table_byte: 0 }
[2022-08-01T09:19:54Z WARN  wasm_mutate::mutators::peephole::dfg] Wasm operator not implemented: CallIndirect { index: 1, table_index: 0, table_byte: 0 }
[2022-08-01T09:19:54Z WARN

[2022-08-01T09:20:26Z WARN  wasm_mutate::mutators::peephole::dfg] Wasm operator not implemented: CallIndirect { index: 0, table_index: 0, table_byte: 0 }
[2022-08-01T09:20:26Z WARN  wasm_mutate::mutators::peephole::dfg] Wasm operator not implemented: CallIndirect { index: 0, table_index: 0, table_byte: 0 }
[2022-08-01T09:20:26Z WARN  wasm_mutate::mutators::peephole::dfg] Wasm operator not implemented: CallIndirect { index: 1, table_index: 0, table_byte: 0 }
[2022-08-01T09:20:26Z WARN  wasm_mutate::mutators::peephole::dfg] Wasm operator not implemented: CallIndirect { index: 1, table_index: 0, table_byte: 0 }
[2022-08-01T09:20:26Z WARN  wasm_mutate::mutators::peephole::dfg] Wasm operator not implemented: CallIndirect { index: 0, table_index: 0, table_byte: 0 }
[2022-08-01T09:20:26Z WARN  wasm_mutate::mutators::peephole::dfg] Wasm operator not implemented: CallIndirect { index: 0, table_index: 0, table_byte: 0 }
[2022-08-01T09:20:26Z WARN  wasm_mutate::mutators::peephole::dfg] Wasm opera

[2022-08-01T10:41:41Z WARN  wasm_mutate::mutators::peephole::dfg] Wasm operator not implemented: CallIndirect { index: 0, table_index: 0, table_byte: 0 }
[2022-08-01T10:41:41Z WARN  wasm_mutate::mutators::peephole::dfg] Wasm operator not implemented: CallIndirect { index: 0, table_index: 0, table_byte: 0 }
[2022-08-01T10:41:48Z WARN  wasm_mutate::mutators::peephole::dfg] Wasm operator not implemented: CallIndirect { index: 1, table_index: 0, table_byte: 0 }
[2022-08-01T10:41:48Z WARN  wasm_mutate::mutators::peephole::dfg] Wasm operator not implemented: CallIndirect { index: 1, table_index: 0, table_byte: 0 }
[2022-08-01T10:41:58Z WARN  wasm_mutate::mutators::peephole::dfg] Wasm operator not implemented: CallIndirect { index: 0, table_index: 0, table_byte: 0 }
[2022-08-01T10:41:58Z WARN  wasm_mutate::mutators::peephole::dfg] Wasm operator not implemented: CallIndirect { index: 0, table_index: 0, table_byte: 0 }
[2022-08-01T10:42:05Z WARN  wasm_mutate::mutators::peephole::dfg] Wasm opera

In [12]:
import json


data = json.loads(open("meta.json", "r").read())


sets = {}
SUM = 0
for m in data['mutations']:
    clname = m['class_name']
    if clname not in sets:
        sets[clname] = 0
    for k, entry in m['generic_map'].items():
        sets[clname] += len(entry)
        SUM += len(entry)
print("How many possible variants", SUM)        
print(sets)
        
    


How many possible variants 162666
{'PeepholeMutator::new(1)': 162549, 'CodemotionMutator': 109, 'AddTypeMutator { max_params: 20, max_results: 20 }': 1, 'AddFunctionMutator': 1, 'RemoveSection::Custom': 2, 'RemoveItemMutator(Item::Function)': 0, 'RemoveItemMutator(Item::Global)': 0, 'RemoveItemMutator(Item::Memory)': 0, 'RemoveItemMutator(Item::Table)': 0, 'RemoveItemMutator(Item::Type)': 0, 'CustomSectionMutator': 4}


Seeing the previous code, the size of the sampling is huge even when the maximum height is 1